In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-13"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6616,2024-07-13,Austrália Nbl1 Norte,06:30,Townsville Heat,Ipswich Force,1.93,1.79,182.5,1.84,1.84,1.5,1.80,1.65,zwDZfJTe,0.518135,0.558659,0.543478,0.543478,0.076794,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,172.66,165.30,0.0,0.0,0.0,0.0,0.053223,0.000000,0.061488,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6617,2024-07-13,Austrália Nbl1 Sul,06:30,Kilsyth,Bendigo Braves,1.83,1.89,158.5,1.80,1.86,-1.5,1.92,2.10,z3C3zwUL,0.546448,0.529101,0.555556,0.537634,0.075549,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,155.40,143.37,0.0,0.0,0.0,0.0,0.022810,0.023184,0.063323,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6618,2024-07-13,Austrália Nbl1 Sul,07:00,Eltham,Ballarat,1.97,1.76,175.5,1.84,1.86,-6.5,1.88,2.29,YwhvUz7k,0.507614,0.568182,0.543478,0.537634,0.075796,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,259.70,0.0,0.0,0.0,0.0,0.079621,0.007644,0.139047,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6619,2024-07-13,Austrália Nbl1 Sul,05:00,Knox F,Keilor Thunder F,1.71,2.03,150.5,1.87,1.79,-2.5,1.95,2.48,Q50wuDmi,0.584795,0.492611,0.534759,0.558659,0.077406,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,106.15,0.0,0.0,0.0,1.0,0.121002,0.030912,0.169195,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6620,2024-07-13,Austrália Nbl1 Central,07:45,Central Districts Lions,South Adelaide Panthers,2.19,1.63,184.5,1.86,1.80,1.5,2.02,1.49,WQ7nK1S3,0.456621,0.613497,0.537634,0.555556,0.070118,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.207319,0.023184,0.213542,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6621,2024-07-13,Chile Lnb 2,21:00,Truenos De Talca,Brisas,1.08,7.06,136.5,1.85,1.85,-14.5,1.87,1.95,CznG1yzD,0.925926,0.141643,0.540541,0.540541,0.067569,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,1.038943,0.000000,0.029617,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
6622,2024-07-13,Eua Nba,17:30,Chicago Bulls,Milwaukee Bucks,1.83,1.95,174.5,1.89,1.89,-1.5,1.93,2.09,AH8BehBc,0.546448,0.512821,0.529101,0.529101,0.059269,0.4,0.6,NaN,NaN,224.990,35.381003,0.157256,339.102,115.307263,0.340037,233.270,314.136,217.46,370.44,0.0,0.0,0.0,0.0,0.044896,0.000000,0.056287,0.86,0.172,4.825581,0.487430,0.4,-0.087430,-5.00,-1.000,-0.950000,0.486347,0.3,-0.186347
6623,2024-07-13,Eua Nba,19:30,Boston Celtics,Miami Heat,2.09,1.72,183.5,1.86,1.86,1.5,1.96,1.53,vc0SiAAM,0.478469,0.581395,0.537634,0.537634,0.059864,0.6,0.8,NaN,NaN,167.632,35.882799,0.214057,262.354,102.788240,0.391792,154.468,239.902,163.59,196.56,0.0,0.0,0.0,0.0,0.137338,0.000000,0.174244,0.87,0.174,6.264368,0.812804,0.8,-0.012804,0.94,0.188,3.829787,0.463344,0.8,0.336656
6624,2024-07-13,Eua Nba,21:30,Toronto Raptors,Oklahoma City Thunder,1.71,2.10,185.5,1.91,1.87,-2.5,1.91,2.37,pEgziqrJ,0.584795,0.476190,0.523560,0.534759,0.060986,0.4,0.2,NaN,NaN,280.112,120.456734,0.430031,212.556,54.821279,0.257915,226.174,215.276,232.05,292.32,0.0,0.0,0.0,0.0,0.144762,0.014965,0.151995,-3.22,-0.644,-1.102484,0.502504,0.2,-0.302504,0.44,0.088,12.500000,0.498260,0.5,0.001740
6625,2024-07-13,Eua Wnba,16:30,Dallas Wings F,Los Angeles Sparks F,1.60,2.36,168.5,1.81,1.95,-3.5,1.92,2.71,jgyaaj7G,0.625000,0.423729,0.552486,0.512821,0.048729,0.0,0.0,NaN,NaN,NaN,NaN,NaN

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,06:30,Austrália Nbl1 Sul,Kilsyth,Bendigo Braves,158.5,1.80,0.9946,Over
1,07:00,Austrália Nbl1 Sul,Eltham,Ballarat,175.5,1.84,1.0000,Over
2,05:00,Austrália Nbl1 Sul,Knox F,Keilor Thunder F,150.5,1.87,1.0000,Over
3,17:30,Eua Nba,Chicago Bulls,Milwaukee Bucks,174.5,1.89,1.0000,Over
4,21:30,Eua Nba,Toronto Raptors,Oklahoma City Thunder,185.5,1.91,1.0000,Over
5,16:30,Eua Wnba,Dallas Wings F,Los Angeles Sparks F,168.5,1.81,1.0000,Over
6,12:00,Europa Eurobasket Sub,França U20 F,Alemanha U20 F,132.5,1.84,1.0000,Over
7,12:00,Europa Campeonato Europeu Sub,Países Baixos U20,Portugal U20,137.5,1.83,1.0000,Over
8,11:00,Mundo Amistosos Internacionais,Portugal F,Porto Rico F,127.5,1.84,0.9986,Over
